In [52]:
#pip install praw plotly wordcloud kaleido statsmodels spacy

In [53]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import praw
import pandas as pd
import time
import re
import plotly
import spacy
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

nlp = spacy.load('es_core_news_sm')


plotly.offline.init_notebook_mode(connected=True) 

# Configura las credenciales de la API de Reddit
reddit = praw.Reddit(client_id='client-id',
                     client_secret='api-key',
                     user_agent='username')

# Especifica el subreddit
subreddit = reddit.subreddit('RepublicadeChile')

# Lista para almacenar los datos
data = []

# Función para controlar la tasa de solicitudes
def rate_limited(max_per_second):
    min_interval = 1.0 / float(max_per_second)
    def decorate(func):
        last_time_called = [0.0]
        def rate_limited_function(*args, **kwargs):
            elapsed = time.time() - last_time_called[0]
            left_to_wait = min_interval - elapsed
            if left_to_wait > 0:
                time.sleep(left_to_wait)
            ret = func(*args, **kwargs)
            last_time_called[0] = time.time()
            return ret
        return rate_limited_function
    return decorate

# Aplicar el decorador a la función que hace las solicitudes
@rate_limited(10)  # 10 solicitudes por segundo
def fetch_reddit_data(submission):
    # Recopila los datos del hilo
    thread_data = {
        "Post ID": submission.id,
        "Título": submission.title,
        "Autor del Post": str(submission.author),
        "Puntuación del Post": submission.score,
        "URL": submission.url,
        "Número de comentarios": submission.num_comments,
        "Texto del Post": submission.selftext,
        "Comentario ID": None,
        "Autor del Comentario": None,
        "Texto del Comentario": None,
        "Puntuación del Comentario": None
    }

    data.append(thread_data)

    # Obtiene los comentarios
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
        comment_data = thread_data.copy()
        comment_data.update({
            "Comentario ID": comment.id,
            "Autor del Comentario": str(comment.author),
            "Texto del Comentario": comment.body,
            "Puntuación del Comentario": comment.score
        })
        data.append(comment_data)

# Itera sobre los hilos
contador=0

#Límite de post que se van a extraer
for submission in subreddit.hot(limit=500):  
    contador+=1
    #print(contador)
    fetch_reddit_data(submission)
    print(f"{contador}. Procesado post: {submission.id}")

# Crea el DataFrame
df = pd.DataFrame(data)

# Muestra las primeras filas del DataFrame

# Guarda el DataFrame en un archivo CSV
df.to_csv('reddit_threads_with_comments_rate_limited_500.csv', index=False)

# Información adicional
print(f"\nNúmero total de filas en el DataFrame: {len(df)}")
print(f"Número de posts únicos: {df['Post ID'].nunique()}")
print(f"Número total de comentarios: {df['Comentario ID'].notna().sum()}")
print(df.head())


' import numpy as np\nimport plotly.graph_objects as go\nimport plotly.express as px\nimport praw\nimport pandas as pd\nimport time\nimport re\nimport plotly\nimport spacy\nfrom collections import Counter\nimport matplotlib.pyplot as plt\nfrom wordcloud import WordCloud,STOPWORDS\n\nnlp = spacy.load(\'es_core_news_sm\')\n\n\nplotly.offline.init_notebook_mode(connected=True) \n\n# Configura las credenciales de la API de Reddit\nreddit = praw.Reddit(client_id=\'DJaJF3LnEdxf-u4cZnvyiw\',\n                     client_secret=\'Zi06iYPwRL1Bpe2wSmlqaZggroRITQ\',\n                     user_agent=\'DifferenceFree738\')\n\n# Especifica el subreddit\nsubreddit = reddit.subreddit(\'RepublicadeChile\')\n\n# Lista para almacenar los datos\ndata = []\n\n# Función para controlar la tasa de solicitudes\ndef rate_limited(max_per_second):\n    min_interval = 1.0 / float(max_per_second)\n    def decorate(func):\n        last_time_called = [0.0]\n        def rate_limited_function(*args, **kwargs):\n       

# Librerías a utilizar en este trabajo

In [54]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import praw
import pandas as pd
import time
import re
import plotly
import spacy
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

nlp = spacy.load('es_core_news_sm')

plotly.offline.init_notebook_mode(connected=True) 

## Carga de los datos

In [55]:
df=pd.read_csv('reddit_threads_with_comments_rate_limited_500.csv')
#df=pd.read_csv('/content/drive/MyDrive/10mo_semestre/Analitica/Trabajo_1/reddit_threads_with_comments_rate_limited_500.csv')
df.columns

Index(['Post ID', 'Título', 'Autor del Post', 'Puntuación del Post', 'URL',
       'Número de comentarios', 'Texto del Post', 'Comentario ID',
       'Autor del Comentario', 'Texto del Comentario',
       'Puntuación del Comentario'],
      dtype='object')

## Funciones para la limpieza de los caracteres

Estas funciones incluyen:
* Eliminación de URLs, menciones y hashtags
* Reemplazo de signos de puntuación y saltos de línea
* Eliminación de caracteres no deseados, manteniendo tíldes y la letra Ñ
* Transformación de texto a minúsculas
* Manejo de datos "nan"
* Eliminación de stopwords
  > Anteriormente se utilizó la librería NTLK para la eliminación de las stopwords pero no hacía un buen trabajo, en cambio la librería spacy hace un mejor trabajo pero se demora mucho más en comparación a NTLK

In [56]:
def limpieza_caracteres(text):

    # Eliminar URLs, menciones y hashtags
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'#\w+', ' ', text)

    # Reemplazar signos de puntuación y saltos de línea
    text = re.sub(r'[,\.\?\!\n\r]+', ' ', text)

    # Eliminar caracteres no deseados, manteniendo tildes
    text = re.sub(r'[^a-zA-ZñÑáéíóúÁÉÍÓÚ0-9\s]+', ' ', text)

    # Convertir a minúsculas
    text = text.lower()

    return text

def limpieza_comentarios(text):
    if text == "nan":
        return "Vacio"

    text = limpieza_caracteres(text)

    # Procesar el texto con spaCy para obtener las stopwords
    doc = nlp(text)
    words = [token.text for token in doc if not token.is_stop]

    return ' '.join(words)

In [60]:
#Eliminación de filas las cuales en la columna "Texto del Comentario" tuviesen valores nulos
df2=df.dropna(subset=['Texto del Comentario'])
print(f"Se redujo de {len(df)} a {len(df2)} comentarios")
print(f"Eliminando así {len(df)-len(df2)} registros")

Se redujo de 18885 a 18385 comentarios
Eliminando así 500 registros


Frecuencia de los comentarios únicos, con esto se buscó hacer una busqueda manual de comentarios autogenerados por bots o comandos escritos por los usuarios, se creó una lista con los índices de los comentarios que estan almacenados en una lista donde se obtuvieron los comentarios únicos y su frecuencia

In [61]:
tempdicc=df2['Texto del Comentario'].value_counts()

In [62]:
aux=tempdicc.keys()

`lista_remover` contiene los índices de las key de un diccionario en el cual se hizo el filtrado de datos, los datos removidos son los siguientes
* '- Estamos escuchando la voz de los usuarios en relacion al influjo de post de cierta tematica [Comenta Aca](https://www.reddit.com/r/RepublicadeChile/comments/1embrfv/ considerando_la_cantidad_de_preguntas/).\n- ¿Tienes historias paranormales que contar o quieres leer relatos extraños y perturbadores? [¡Visita nuestro thread paranormal!](https://www.reddit.com/r/RepublicadeChile/comments/1e6a9rs/historias_paranormales_que_nadie_te_creer%C3%ADa/).\n- ¿Quieres compartir a tu michi, perro  u otro animal exótico? [¡Visita nuestro hilo de las mascotas del Sub!](https://www.reddit.com/r/RepublicadeChile/comments/1e6akt6/hilo_de_las_mascotas_del_sub/).\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/RepublicadeChile) if you have any questions or concerns.*'
* '- ¿Tienes historias paranormales que contar o quieres leer relatos extraños y perturbadores? [¡Visita nuestro thread paranormal!](https://www.reddit.com/r/RepublicadeChile/comments/1e6a9rs/historias_paranormales_que_nadie_te_creer%C3%ADa/).\n- ¿Quieres compartir a tu michi, perro  u otro animal exótico? [¡Visita nuestro hilo de las mascotas del Sub!](https://www.reddit.com/r/RepublicadeChile/comments/1e6akt6/hilo_de_las_mascotas_del_sub/).\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/RepublicadeChile) if you have any questions or concerns.*'
* '[deleted]'
* '!remindme 48 hours'
* '[removed]'
* '!remindme 24 hours'
* '!remindme 16 hours'
* '!remindme 20 hours'
* '!remindme 40 hours'
* '!remindme 12 hours'
* '!remindme 41 hours'


In [63]:
lista_remover=[0,1,2,3,4,8,42,85,99,1500,1537]

In [64]:
listaxd=[]
for i in range(len(aux)):
    if i in lista_remover:
        listaxd.append(aux[i])
    #print(f"{i}:  {aux[i]}")

In [65]:
# Eliminamos las filas que contengan los valores de la lista_filtros en la columna 'Texto del comentario'
df4 = df[~df['Texto del Comentario'].isin(listaxd)]

In [66]:
df4.reset_index(inplace=True)
df4=df4.dropna(subset=['Texto del Comentario'])
df4.reset_index(inplace=True)

In [67]:
#Ejemplo de comentario limpio vs comentario sin limpiar
limpieza_caracteres(df4['Texto del Comentario'][2]), df4['Texto del Comentario'][2]

(' alguien que sea fanático de la banda japonesa  fear  and loathing in las vegas  ',
 '¿Alguien que sea fanático de la banda japonesa "Fear, and Loathing in Las Vegas"?')



Se crearon 2 listas: 
* `comentarios`: Lista anidada que contiene cada comentarios limpio separado
* `listacompleta`: Lista de una dimensión que contiene todas las palabras de los comentarios limpios, le intención de esta lista era luego contar la frecuencia de las palabras y poder graficar esos resultados

In [68]:
comentarios=[]
listacompleta=[]
for i in range(len(df4)):
    temp=limpieza_comentarios(df4['Texto del Comentario'][i]).split()
    listacompleta.extend(temp)
    comentarios.append(temp)

In [123]:
''.join(comentarios[0])

'luznoticiasrecientesencuentrodeberíamosorganizarnospaísenfocarnosrrsspacarbonearincitarlíderesconfrontamientosfísicosadversariosusarlaspadiscutamos'

In [124]:
largo=len(comentarios)
cont=0.0
cont1=0.0
for i in range(largo):
    cont+=len(comentarios[i])
    cont1+=len(''.join(comentarios[i]))
    
print(f"En promedio los comentarios tienen {cont/largo} palabras")
print(f"En promedio los comentarios tienen {cont1/largo} caracteres")


En promedio los comentarios tienen 14.22248467465272 palabras
En promedio los comentarios tienen 91.18041729936449 caracteres


In [69]:
# Función para contar y ordenar elementos en una lista
def count_and_sort(my_list):
    # Cuenta la frecuencia de cada elemento en la lista
    count = Counter(my_list)
    
    # Ordena los elementos por su frecuencia en orden descendente
    sorted_count = sorted(count.items(), key=lambda x: x[1], reverse=True)
    
    # Retorna la lista de elementos ordenados junto con su frecuencia
    return sorted_count

# Función para generar y guardar una nube de palabras
def generate_wordcloud(my_list, nombre_archivo):
    # Llama a la función count_and_sort para contar y ordenar la lista
    counted_items = count_and_sort(my_list)

    # Convierte la lista de tuplas (palabra, frecuencia) en un diccionario
    word_dict = dict(counted_items)

    # Genera la nube de palabras usando las frecuencias de las palabras
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_dict)

    # Configura la figura para mostrar la nube de palabras
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # Elimina los ejes
    plt.tight_layout(pad=0)
    
    # Guarda la imagen de la nube de palabras en un archivo con nombre especificado
    plt.savefig(nombre_archivo+"WC.png", dpi=300)
    
    # Muestra la nube de palabras en pantalla
    plt.show()

In [101]:
df4=df4.sort_values(by='Número de comentarios', ascending=False)
df4.reset_index(inplace=True, drop=True)
dffinal=[]

for i in df4['Título'].unique():
    # Selecciona las filas correspondientes a un título específico
    temp = df4[df4['Título'] == i].copy()  # Usa .copy() para evitar el SettingWithCopyWarning
    
    # Asigna el número de comentarios reales para cada fila
    temp.loc[:, 'Número real de comentarios'] = len(temp)
    
    # Agrega el DataFrame temporal a la lista
    dffinal.append(temp)

dffinal=pd.concat(dffinal)
dffinal.reset_index(inplace=True, drop=True)

In [102]:
idx=0
dffinal['Título'][idx], dffinal['Número de comentarios'][idx], dffinal['Número real de comentarios'][idx]

('Pillé a mi mamá en Tinder… ', 732, 487)

In [103]:

def plotly_graph(df, titulo, limite, nombre_archivo):   
    comentarios=[]
    listacompletaTPP=[]
    for i in range(len(df)):
        temp=limpieza_comentarios(df['Texto del Comentario'][i]).split()
        listacompletaTPP.extend(temp)
        comentarios.append(temp)
    
    counted_items = count_and_sort(listacompletaTPP)
    # Create a dictionary from the counted items
    word_freq = dict(counted_items)

    # Ordenar el diccionario por frecuencia de mayor a menor y tomar las primeras 25 palabras
    sorted_word_freq = dict(sorted(word_freq.items(), key=lambda item: item[1], reverse=True)[:limite])

    # Crear las listas para el eje y (palabras) y el eje x (frecuencias)
    words = list(sorted_word_freq.keys())
    frequencies = list(sorted_word_freq.values())

    fig = go.Figure(go.Bar(
        x=words,
        y=frequencies,
        text=frequencies,
        textposition='outside',
        marker_color='skyblue',
        marker_line_color='rgb(8,48,107)',
        marker_line_width=1.5
    ))

    # Personalizar el diseño
    fig.update_layout(
        title={
            'text': f"Top {limite} Palabras Más Frecuentes del post: {titulo}",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='Palabras',
        yaxis_title='Frecuencia',
        height=600,
        width=1000,
        margin=dict(l=50, r=20, t=70, b=120),
        xaxis=dict(tickangle=45),
        yaxis=dict(zeroline=False, gridwidth=2),
        bargap=0.2
    )

    # Ajustar el rango del eje y para dar espacio a las etiquetas de texto
    max_freq = max(frequencies)
    fig.update_yaxes(range=[0, max_freq * 1.1])
    fig.write_image(nombre_archivo+".png", scale=10)

    # Mostrar el gráfico
    fig.show()

In [119]:

def totallargo(valor):
    unicos=df4['Título'].unique()
    cont=0
    for i in range(valor,0,-1):
       largo=df4[df4['Título']==unicos[0]] 
       cont+=len(largo)
    
    return cont

df4['Título'][totallargo(2)]

'Tier List de Comida rápida en Chile '

In [120]:

def tendencias_por_post(df, nombre_archivo):
  comentarios=[]
  listacompletaTPP=[]
  for i in range(len(df)):
      temp=limpieza_comentarios(df['Texto del Comentario'][i]).split()
      listacompletaTPP.extend(temp)
      comentarios.append(temp)

  generate_wordcloud(listacompletaTPP, nombre_archivo)
  #print(comentarios[0])

titulo_unico=df4['Título'][totallargo(10)]#1,2, 10,15,
dftest=df4[df4['Título']==titulo_unico]
dftest.reset_index(inplace=True, drop=True)
nombre_archivo=titulo_unico
#tendencias_por_post(dftest, limpieza_comentarios(nombre_archivo))
plotly_graph(dftest, titulo_unico, 20, limpieza_comentarios(nombre_archivo))

# Análisis métricas

In [106]:
dffinal['Puntuación del Post'].describe()

count    17781.000000
mean       138.929475
std        243.674248
min          0.000000
25%         11.000000
50%         55.000000
75%        161.000000
max       1354.000000
Name: Puntuación del Post, dtype: float64

In [128]:
correlacion = dffinal[['Puntuación del Post', 'Número de comentarios']].corr()

# Gráfico de dispersión
fig = px.scatter(dffinal, 
                 x='Puntuación del Post', 
                 y='Número real de comentarios', 
                 title='Relación entre Puntuación del Post y Número de Comentarios',
                 labels={'Puntuación del Post': 'Puntuación del Post', 'Número de comentarios': 'Número de Comentarios'},
                 trendline='ols')  # Agrega una línea de tendencia
fig.write_image("relacion puntuacion post y numero de comentarios.png", scale=10)
fig.update_layout(width=800, height=600)
fig.show()


In [129]:

# Conteo de comentarios por usuario
usuarios_activos = dffinal.groupby('Autor del Comentario')['Comentario ID'].count().reset_index()
usuarios_activos = usuarios_activos.rename(columns={'Comentario ID': 'Número de Comentarios'})
usuarios_activos = usuarios_activos.sort_values(by='Número de Comentarios', ascending=False)

# Top 10 usuarios más activos
top_usuarios = usuarios_activos.head(10)

# Gráfico de barras
fig = px.bar(top_usuarios, 
             x='Autor del Comentario', 
             y='Número de Comentarios', 
             title='Top 10 Usuarios Más Activos',
             labels={'Autor del Comentario': 'Usuario', 'Número de Comentarios': 'Número de Comentarios'},
             text='Número de Comentarios')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(width=800, height=600)
fig.write_image("top 10 usuarios mas activos.png", scale=10)
fig.show()


In [136]:
# Puntuación promedio por usuario
puntuacion_usuarios = df.groupby('Autor del Comentario')['Puntuación del Comentario'].mean().reset_index()
puntuacion_usuarios = puntuacion_usuarios.rename(columns={'Puntuación del Comentario': 'Puntuación Promedio'})

# Unir datos de actividad y puntuación
usuarios_analisis = usuarios_activos.merge(puntuacion_usuarios, on='Autor del Comentario')
# Gráfico de dispersión
fig = px.scatter(usuarios_analisis, 
                 x='Número de Comentarios', 
                 y='Puntuación Promedio', 
                 title='Relación entre Número de Comentarios y Puntuación Promedio de Usuarios',
                 labels={'Número de Comentarios': 'Número de Comentarios', 'Puntuación Promedio': 'Puntuación Promedio'},
                 hover_data=['Autor del Comentario'],  # Agrega el nombre del usuario en el hover
                 trendline='ols')  # Agrega una línea de tendencia
fig.update_layout(width=800*0.9, height=600*0.9)
fig.write_image("relacion entre comentarios y puntuacion.png", scale=10)
fig.show()


In [111]:
var=usuarios_analisis.sort_values(by='Puntuación Promedio')
var

,Autor del Comentario,Número de Comentarios,Puntuación Promedio
2294,mangandini,1,-28.0
1045,arianyscrea,4,-20.0
3269,IligalJota,1,-16.0
4395,One-Incident-8915,1,-14.0
3598,Aleex666_,1,-14.0
...,...,...,...
1801,tatemblando,2,240.0
3212,EnLaPasta,1,329.0
4231,Markov_DG,1,353.0
4506,NXRTHNXRTHPVNK,1,425.0


In [112]:
dffinal['Número real de comentarios'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 17781 entries, 0 to 17780
Series name: Número real de comentarios
Non-Null Count  Dtype
--------------  -----
17781 non-null  int64
dtypes: int64(1)
memory usage: 139.0 KB


In [113]:
def avg_num_com(dffinal):
    cont=0
    for i in dffinal['Título'].unique():
        temp=dffinal[dffinal['Título']==i]
        temp.reset_index(inplace=True,drop=True)
        cont+=int(temp['Número real de comentarios'][0])
    print(f"En promedio hay {cont/len(dffinal['Título'].unique())} comentarios por post")
avg_num_com(dffinal)

En promedio hay 38.321120689655174 comentarios por post


In [114]:
print(dffinal[dffinal['Autor del Comentario']=='NXRTHNXRTHPVNK']['Texto del Comentario'])

1288    es simple, en su momento fuiste la mejor opció...
Name: Texto del Comentario, dtype: object


In [127]:
largo=len(dffinal)
cont=0.0
cont1=0.0
for i in range(largo):
    cont+=len(dffinal['Texto del Comentario'][i].split())
    cont1+=len(''.join(dffinal['Texto del Comentario'][i]))
    
print(f"En promedio los comentarios tienen {cont/largo} palabras")
print(f"En promedio los comentarios tienen {cont1/largo} caracteres")


En promedio los comentarios tienen 35.383330521343005 palabras
En promedio los comentarios tienen 195.41938023733198 caracteres
